In [11]:
import dgl
from dgl import DGLGraph
from dgl.data import citegrh
from dgl.nn.pytorch import conv

# Load Pytorch as backend
dgl.load_backend('pytorch')

In [12]:
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F

In [13]:
class GraphSAGE(nn.Module):
    def __init__(self,
                 g,
                 in_feats,
                 n_hidden,
                 n_classes,
                 n_layers,
                 activation,
                 dropout,
                 aggregator_type):
        super(GraphSAGE, self).__init__()
        self.layers = nn.ModuleList()
        self.g = g

        # input layer
        self.layers.append(conv.SAGEConv(in_feats, n_hidden, aggregator_type, feat_drop=dropout, activation=activation))
        # hidden layers
        for i in range(n_layers - 1):
            self.layers.append(conv.SAGEConv(n_hidden, n_hidden, aggregator_type, feat_drop=dropout, activation=activation))
        # output layer
        self.layers.append(conv.SAGEConv(n_hidden, n_classes, aggregator_type, feat_drop=dropout, activation=None)) # activation None

    def forward(self, features):
        h = features
        for layer in self.layers:
            h = layer(self.g, h)
        return h


## Load data

In [14]:
# load and preprocess dataset
data = citegrh.load_pubmed()
features = torch.FloatTensor(data.features)
labels = torch.LongTensor(data.labels)
train_mask = torch.ByteTensor(data.train_mask)
val_mask = torch.ByteTensor(data.val_mask)
test_mask = torch.ByteTensor(data.test_mask)
in_feats = features.shape[1]
n_classes = data.num_labels
n_edges = data.graph.number_of_edges()
print("""----Data statistics------'
      #Edges %d
      #Classes %d
      #Train samples %d
      #Val samples %d
      #Test samples %d""" %
          (n_edges, n_classes,
           train_mask.sum().item(),
           val_mask.sum().item(),
           test_mask.sum().item()))

Finished data loading and preprocessing.
  NumNodes: 19717
  NumEdges: 88651
  NumFeats: 500
  NumClasses: 3
  NumTrainingSamples: 60
  NumValidationSamples: 500
  NumTestSamples: 1000
----Data statistics------'
      #Edges 88651
      #Classes 3
      #Train samples 60
      #Val samples 500
      #Test samples 1000


In [15]:
g = data.graph
print(g.number_of_nodes(), g.number_of_edges())
g.remove_edges_from(g.selfloop_edges())
print(g.number_of_nodes(), g.number_of_edges())
g = DGLGraph(g)
g.readonly()

19717 88651
19717 88648


## Hyperparameters

In [16]:
lr = 2e-3
dropout = 0.5
aggregator_type = 'gcn'
weight_decay = 5e-4
n_epochs = 200

## Node classification task

In [17]:
# Hyperparameters
n_hidden = 64
n_layers = 2

# create GraphSAGE model
gconv_model = GraphSAGE(g,
                        in_feats,
                        n_hidden,
                        n_classes,
                        n_layers,
                        F.relu,
                        dropout,
                        aggregator_type)

class NodeClassification(nn.Module):
    def __init__(self, gconv_model):
        super(NodeClassification, self).__init__()
        self.gconv_model = gconv_model
        self.loss_fcn = torch.nn.CrossEntropyLoss()

    def forward(self, features):
        logits = self.gconv_model(features)
        return self.loss_fcn(logits[train_mask], labels[train_mask])

# Node classification task
model = NodeClassification(gconv_model)
    
def evaluate(model, features):
    model.eval()
    with torch.no_grad():
        logits = gconv_model(features)
        logits = logits[test_mask]
        test_labels = labels[test_mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == test_labels)
        return correct.item() * 1.0 / len(test_labels)

## Link prediction

We first split the graph into the training set and the testing set.

In [ ]:
eids = np.random.permutation(g.number_of_edges())
train_eids = eids[:int(len(eids) * 0.8)]
test_eids = eids[int(len(eids) * 0.8):]
train_g = g.edge_subgraph(train_eids, preserve_nodes=True)
test_g = g.edge_subgraph(test_eids, preserve_nodes=True)

Construct negative edges for training

In [ ]:
def neg_sample(g, neg_sample_size, edges=None, return_false_neg=True):
    sampler = dgl.contrib.sampling.EdgeSampler(g, batch_size=int(g.number_of_edges()/10),
                                               seed_edges=edges,
                                               neg_sample_size=neg_sample_size,
                                               negative_mode='tail',
                                               shuffle=True,
                                               return_false_neg=return_false_neg)
    sampler = iter(sampler)
    return next(sampler)

In [ ]:
#hyperparameters
n_hidden = 64
neg_sample_size = 100
n_layers = 1

#np.set_printoptions(edgeitems=100)

# create GraphSAGE model
gconv_model = GraphSAGE(train_g,
                        in_feats,
                        n_hidden,
                        n_hidden,
                        n_layers,
                        F.relu,
                        dropout,
                        aggregator_type)

def pos_score_func(g, emb):
    pos_src, pos_dst = g.all_edges()
    pos_heads = emb[g.parent_nid[pos_src]]
    pos_tails = emb[g.parent_nid[pos_dst]]
    return torch.sum(pos_heads * pos_tails, dim=1)

def neg_score_func(g, emb):
    neg_src, neg_dst = g.all_edges()
    neg_heads = emb[g.parent_nid[neg_src]].reshape(-1, neg_sample_size, emb.shape[1])
    neg_tails = emb[g.parent_nid[neg_dst]].reshape(-1, neg_sample_size, emb.shape[1])
    assert neg_heads.shape[0] == neg_tails.shape[0]
    return torch.sum(neg_heads * neg_tails, dim=2)

class LinkPrediction(nn.Module):
    def __init__(self, gconv_model):
        super(LinkPrediction, self).__init__()
        self.gconv_model = gconv_model

    def forward(self, features):
        emb = self.gconv_model(features)
        pos_g, neg_g = neg_sample(g, neg_sample_size, return_false_neg=False)
        pos_score = pos_score_func(pos_g, emb)
        neg_score = neg_score_func(neg_g, emb)
        pos_score = F.logsigmoid(pos_score)
        neg_score = F.logsigmoid(-neg_score)
        return torch.mean(-pos_score - torch.sum(neg_score, dim=1))
    

# Link prediction task
model = LinkPrediction(gconv_model)

def evaluate(model, features):
    model.eval()
    with torch.no_grad():
        emb = gconv_model(features)
        
        pos_g, neg_g = neg_sample(g, neg_sample_size, test_eids, return_false_neg=True)
        pos_score = pos_score_func(pos_g, emb)
        neg_score = neg_score_func(neg_g, emb)
        filter_bias = neg_g.edata['false_neg'].reshape(-1, neg_sample_size)

        pos_score = F.logsigmoid(pos_score)
        neg_score = F.logsigmoid(neg_score)
        neg_score -= filter_bias.float()
        pos_score = pos_score.unsqueeze(1)
        rankings = torch.sum(neg_score > pos_score, dim=1) + 1
        print('MR:', np.mean(rankings.cpu().numpy()))
        print('MRR:', np.mean(1.0/rankings.cpu().numpy()))
        return np.mean(1.0/rankings.cpu().numpy())

The training loop

In [18]:
# use optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

# initialize graph
dur = []
for epoch in range(n_epochs):
    model.train()
    if epoch >= 3:
        t0 = time.time()
    # forward
    loss = model(features)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch >= 3:
        dur.append(time.time() - t0)

    acc = evaluate(model, features)
    print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | Accuracy {:.4f} | "
            "ETputs(KTEPS) {:.2f}".format(epoch, np.mean(dur), loss.item(),
                                            acc, n_edges / np.mean(dur) / 1000))

print()
acc = evaluate(model, features)
print("Test Accuracy {:.4f}".format(acc))


/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/usr/local/lib/python3.5/dist-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.5/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encounter

Epoch 00000 | Time(s) nan | Loss 1.0922 | Accuracy 0.4630 | ETputs(KTEPS) nan
Epoch 00001 | Time(s) nan | Loss 1.0940 | Accuracy 0.5930 | ETputs(KTEPS) nan


/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype

Epoch 00002 | Time(s) nan | Loss 1.0887 | Accuracy 0.6330 | ETputs(KTEPS) nan
Epoch 00003 | Time(s) 0.1253 | Loss 1.0755 | Accuracy 0.6500 | ETputs(KTEPS) 707.79


/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype

...

/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.


Epoch 00197 | Time(s) 0.1398 | Loss 0.0258 | Accuracy 0.7720 | ETputs(KTEPS) 633.92


/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.


Epoch 00198 | Time(s) 0.1399 | Loss 0.0456 | Accuracy 0.7680 | ETputs(KTEPS) 633.61
Epoch 00199 | Time(s) 0.1400 | Loss 0.0307 | Accuracy 0.7690 | ETputs(KTEPS) 633.04

Test Accuracy 0.7690


/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
